In [1]:
import os
project_path = 'C://Users//choi1//Downloads//CmBERT'
os.environ["HF_HOME"] = project_path
import sys
sys.path.append(project_path)
from os.path import join
import torch
import torch.nn as nn
from model import Seq2Seq
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    get_linear_schedule_with_warmup,
    RobertaConfig,
    RobertaModel,
    RobertaTokenizer,
)
import re

c:\Users\choi1\Downloads\CmBERT\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
model_path = join(project_path, "output//python//20231028_204326//checkpoint-best-bleu//pytorch_model.bin")
model_name = "microsoft/codebert-base"

In [35]:
config = RobertaConfig.from_pretrained(model_name)
encoder = RobertaModel.from_pretrained(model_name, config=config)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
decoder_layer = nn.TransformerDecoderLayer(
    d_model=config.hidden_size, nhead=config.num_attention_heads
)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model = Seq2Seq(
    encoder=encoder,
    decoder=decoder,
    config=config,
    beam_size=10,
    max_length=128,
    sos_id=tokenizer.cls_token_id,
    eos_id=tokenizer.sep_token_id,
).to("cuda")
model.load_state_dict(torch.load(model_path), strict=False)
model.eval()

Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [36]:
def process_fn(array):
        return re.sub("[Ġ]{2,}", "", "".join(array)).replace("Ġ", " ")

In [37]:
added = ["import", "Ġio", ".", "qu", "ark", "us", ".", "vert", "x", ".", "core", ".", "de", "ploy", "ment", ".", "Event", "Consumer", "Business", "Method", "Item", ";", "import", "Ġio", ".", "qu", "ark", "us", ".", "vert", "x", ".", "core", ".", "de", "ploy", "ment", ".", "Internal", "Vert", "x", "Build", "Item", ";"]
deleted = ["import", "Ġio", ".", "qu", "ark", "us", ".", "vert", "x", ".", "common", ".", "de", "ploy", "ment", ".", "Event", "Consumer", "Business", "Method", "Item", ";", "import", "Ġio", ".", "qu", "ark", "us", ".", "vert", "x", ".", "common", ".", "de", "ploy", "ment", ".", "Internal", "Vert", "x", "Build", "Item", ";"]

In [38]:
input = tokenizer(process_fn(added))
source_ids = torch.tensor(input["input_ids"]).unsqueeze(0).to("cuda")
source_mask = torch.tensor(input["attention_mask"]).unsqueeze(0).to("cuda")

In [39]:
with torch.no_grad():
    with torch.cuda.amp.autocast():
        preds = model(source_ids=source_ids, source_mask=source_mask)
    for pred in preds:
        t = pred[0].cpu().numpy()
        t = list(t)
        if 0 in t:
            t = t[: t.index(0)]
        text = tokenizer.decode(t, clean_up_tokenization_spaces=False)
        print(text)

Rename EventConsumerItem to EventConsumer 


In [58]:
def run_model(added: str, deleted: str):
    input = tokenizer(
        text=added + " " + deleted,
        return_tensors="pt",
    )
    # print(tokenizer.decode(input["input_ids"][0]))
    input = {k: v.to("cuda") for k, v in input.items()}
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            preds = model(
                source_ids=input["input_ids"],
                source_mask=input["attention_mask"],
            )
        for pred in preds:
            t = pred[0].cpu().numpy()
            t = list(t)
            # print(t)
            if 0 in t:
                t = t[: t.index(0)]
            text = tokenizer.decode(t, clean_up_tokenization_spaces=False)
            print(text)


In [67]:
added = "def add(a, b): return a + b"
deleted = "def sub(a, b): return a - b"
run_model(added, deleted)

change sub to add


In [60]:
added = "from pandas.util.compat import zip"
deleted = "from pandas.util.py3compat import zip"
run_model(added, deleted)

CLN/ENH: Merge compat with py3compat and six


In [61]:
added = "return output_errors if squared else np.sqrt(output_errors)"
deleted = "return output_errors"
run_model(added, deleted)

Internal change


In [62]:
added = "def log(x): return np.log(x)"
deleted = "def exp(x): return np.exp(x)"
run_model(added, deleted)

Cosmetics


In [68]:
added = "import OpenSSL"
deleted = "import OpenSSL.cryptoimport OpenSSL.SSL"
run_model(added, deleted)

Remove unused imports


In [69]:
added = "max_length = tokenizer.model_max_length"
deleted = "max_length = tokenizer.max_len"
run_model(added, deleted)

Merged commit includes the following changes: 


In [70]:
added = "import pandas as pd n_hidden = 50 n_iteractions = 20000 learning_rate = 0.01"
deleted = "n_hidden = 20 n_iteractions = 80000 learning_rate = 0.001"
run_model(added, deleted)

[MRG+1] Increase learning rate for SGD 
